# Requirement 4

In [ ]:
# Import
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import stats
import random

In [ ]:
# utility function
def solve_linear_program(f, ct, rho):
    c = -f
    A_ub = [ct]
    b_ub = [rho]
    A_eq = [np.ones(len(f))]
    b_eq = [1]
    res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0,1))
    return res.x, -res.fun

def find_rank(bid, m_t):
    for i in range(len(m_t)):
        if bid >= m_t[i]:
            return i
    return -1


def find_index(winners, i):
    for j in range(len(winners)):
        if winners[j] == i:
            return j
    return -1

## Creating the Auction class

In [ ]:
class Auction:
    def __init__(self, click_through_rates, *args, **kwargs):
        pass

    def get_winners(self, bids):
        pass

    def get_payments_per_click(self, winners, values, bids):
        pass

    def round(self, bids):
        winners, values = self.get_winners(bids) # allocation mechanism
        payments_per_click = self.get_payments_per_click(winners, values, bids) # payment rule
        return winners, payments_per_click
    
    def get_click_through_rates(self):
        return self.click_through_rates


class GeneralizedFirstPriceAuction(Auction):
    def __init__(self, click_through_rates, lambdas):
        self.click_through_rates = click_through_rates
        self.lambdas = lambdas
        self.n_advertisers = len(self.click_through_rates)
        self.n_slots = len(self.lambdas)
    
    def get_winners(self, bids):
        advertisers_values = self.click_through_rates * bids
        advertisers_ranking = np.argsort(advertisers_values)
        winners = advertisers_ranking[-self.n_slots:]
        winners = winners[::-1]
        winners_values = advertisers_values[winners]
        return np.array(winners), winners_values
    
    def get_payments_per_click(self, winners, values, bids):
        payments_per_click = np.array(bids)[winners]
        return [payment.round(2) for payment in payments_per_click]

## Creating Agents for the assignment

In [ ]:
# Virtual class for the bidding agents
class BiddingAgent:
    def __init__(self):
        pass

    def bid(self):
        pass

    def update(self, lambdas, slot, c_t, m_t=None):
        pass

    def get_utility(self):
        return np.array(self.utility).sum()

    def get_log(self):
        return self.log_bids, self.log_slots

In [ ]:
# Classic UCB Bidding Agent 
class UCBBiddingAgentExpert(BiddingAgent):
    def __init__(self, valuation, budget, T):
        self.available_bids = np.linspace(0,valuation,101)
        self.valuation = valuation
        self.budget = budget
        self.T = T
        self.rho = self.budget/self.T
        self.action_t = None
        self.n_pulls = np.zeros(len(self.available_bids))
        self.t = 0

        self.utility = []
        self.log_win = np.zeros(5)
        self.log_bids = []
        self.log_slots = []
        self.average_utilities = np.zeros(len(self.available_bids))
        self.average_costs = np.zeros(len(self.available_bids))

        self.name = "UCB_classic " +str(self.valuation)
        self.color = "red"

    def get_name(self):
        return self.name

    def get_color(self):
        return self.color

    def bid(self):
        if self.budget < 1:
            self.action_t = 0

        elif self.t == 0:
            self.action_t = np.random.choice(range(len(self.available_bids)))

        else:
            # no need to add exploration term because of expert feedback
            ucb_utility_values = self.average_utilities  #+np.sqrt(2*np.log(self.t)/self.t)
            ucb_cost_values = self.average_costs #-np.sqrt(2*np.log(self.t)/self.t)
            gamma, fun = solve_linear_program(ucb_utility_values, ucb_cost_values, self.rho)
            self.action_t = np.random.choice(range(len(self.available_bids)), p=gamma)

        self.log_bids.append(self.available_bids[self.action_t])
        return self.available_bids[self.action_t]

    def update(self, lambdas, slot, c_t, m_t=None):
        self.t += 1
        for i, b in enumerate(self.available_bids):
            rank = find_rank(b, m_t)
            if rank != -1:
                f = (self.valuation-b)*lambdas[rank]
                c = b
            else:
                f = c = 0
            self.average_utilities[i] += (f - self.average_utilities[i]) / self.t
            self.average_costs[i] += (c - self.average_costs[i]) / self.t

        f_t = 0
        if slot != -1:
            f_t = self.valuation*lambdas[slot]-c_t

        self.utility.append(f_t)
        self.budget -= c_t
        self.log_slots.append(slot)

    def get_utility(self):
        return np.array(self.utility).sum()

    def get_log(self):
        return self.log_bids, self.log_slots

In [ ]:
# Our take on UCB Bidding Agent (see presentation for more)
class UCBBiddingAgentExpertUpdateRho(BiddingAgent):
    def __init__(self, valuation, budget, T):
        self.available_bids =  np.linspace(0,valuation,101)
        self.valuation = valuation
        self.budget = budget
        self.T = T
        self.rho = self.budget/self.T
        self.action_t = None
        self.n_pulls = np.zeros(len(self.available_bids))
        self.t = 0

        self.utility = []
        self.log_win = np.zeros(5)
        self.log_bids = []
        self.log_slots = []
        self.average_utilities = np.zeros(len(self.available_bids))
        self.average_costs = np.zeros(len(self.available_bids))

        self.name = "UCB_rho " +str(self.valuation)
        self.color = "orange"

    def get_name(self):
        return self.name

    def get_color(self):
        return self.color

    def bid(self):
        if self.budget < 1:
            self.action_t = 0

        elif self.t == 0:
            self.action_t = np.random.choice(range(len(self.available_bids)))

        else:
            ucb_utility_values = self.average_utilities
            ucb_cost_values = self.average_costs
            self.rho = self.budget/ (self.T-self.t)  # our idea is to update rho at every step
            gamma, fun = solve_linear_program(ucb_utility_values, ucb_cost_values, self.rho)
            self.action_t = np.random.choice(range(len(self.available_bids)), p=gamma)

        self.log_bids.append(self.available_bids[self.action_t])
        return self.available_bids[self.action_t]

    def update(self, lambdas, slot, c_t, m_t=None):
        self.t += 1
        for i, b in enumerate(self.available_bids):
            rank = find_rank(b, m_t)
            if rank != -1:
                f = (self.valuation-b)*lambdas[rank]
                c = b
            else:
                f = c = 0
            self.average_utilities[i] += (f - self.average_utilities[i]) / self.t
            self.average_costs[i] += (c - self.average_costs[i]) / self.t

        f_t = 0
        if slot != -1:
            f_t = self.valuation*lambdas[slot]-c_t

        self.utility.append(f_t)
        self.budget -= c_t
        self.log_slots.append(slot)

    def get_utility(self):
        return np.array(self.utility).sum()

    def get_log(self):
        return self.log_bids, self.log_slots

In [ ]:
class MultiplicativePacingBiddingAgent(BiddingAgent):
    def __init__(self, valuation, budget, T, learning_rate=0.1):
        self.valuation = valuation
        self.budget = budget
        self.T = T
        self.learning_rate = learning_rate
        self.rho = self.budget/self.T
        self.lmbd = 1
        self.t = 0
        self.utility = []

        self.log_bids = []
        self.log_slots = []

        self.name = "Multiplicative " +str(self.valuation)
        self.color = "blue"

    def get_name(self):
        return self.name

    def get_color(self):
        return self.color

    def bid(self):
        if self.budget < 1:
            return 0

        self.log_bids.append(self.valuation/(self.lmbd+1))
        return self.valuation/(self.lmbd+1)

    def update(self, lambdas, slot, c_t, m_t=None):
        self.lmbd = np.clip(self.lmbd-self.learning_rate*(self.rho-c_t), a_min=0, a_max=1/self.rho)
        self.budget -= c_t
        if slot != -1:
            self.utility.append(self.valuation*lambdas[slot]-c_t)
        else:
            self.utility.append(0)
        self.log_slots.append(slot)

    def get_utility(self):
        return np.array(self.utility).sum()

    def get_log(self):
        return self.log_bids, self.log_slots

In [ ]:
class HedgeAgent:
    def __init__(self, K, learning_rate):
        self.K = K
        self.learning_rate = learning_rate
        self.weights = np.ones(K)
        self.x_t = np.ones(K)/K         # Normalized weights (uniform distribution)
        self.action_t = None
        self.t = 0

    def pull_arm(self):
        self.x_t = self.weights / self.weights.sum()
        self.action_t = np.random.choice(np.arange(self.K), p=self.x_t)
        return self.action_t

    def update(self, loss_t):
        self.weights *= np.exp(-self.learning_rate * loss_t)
        self.t += 1


class FFMultiplicativePacingBiddingAgent(BiddingAgent):
    def __init__(self, valuation, budget, T, learning_rate=0.1):
        self.bids_set = np.linspace(0,valuation,101)
        self.K = len(self.bids_set)
        self.hedge = HedgeAgent(self.K, np.sqrt(np.log(self.K)/T))
        self.valuation = valuation
        self.budget = budget
        self.T = T
        self.learning_rate = learning_rate
        self.rho = self.budget / self.T
        self.lmbd = 1
        self.t = 0
        self.utility = []

        self.log_bids = []
        self.log_slots = []

        self.name = "FF_multiplicative " +str(self.valuation)
        self.color = "green"

    def get_name(self):
        return self.name

    def get_color(self):
        return self.color

    def bid(self):
        if self.budget < 1:
            return 0

        self.log_bids.append(self.bids_set[self.hedge.pull_arm()])
        return self.bids_set[self.hedge.pull_arm()]

    def update(self, lambdas, slot, c_t, m_t):
        f_t_full = np.zeros(len(self.bids_set))
        c_t_full = np.zeros(len(self.bids_set))
        for i,b in enumerate(self.bids_set):
            rank = find_rank(b, m_t)
            if rank != -1:
                f_t_full[i] = (self.valuation-b)*lambdas[rank]
                c_t_full[i] = b*lambdas[rank]
            else:
                f_t_full[i] = c_t_full[i] = 0

        L = f_t_full-self.lmbd*(c_t_full-self.rho)
        L_range = np.max(L)-np.min(L)
        if L_range < 1e-8:
            L_range = 1
        self.hedge.update(1+(np.min(L)-L)/L_range) # Hedge needs losses in [0,1]

        self.lmbd = np.clip(self.lmbd-self.learning_rate*(self.rho-c_t), a_min=0, a_max=1/self.rho)
        self.budget -= c_t
        self.t += 1

        if slot != -1:
            self.utility.append(self.valuation*lambdas[slot] - c_t)
        else:
            self.utility.append(0)

        self.log_slots.append(slot)

    def get_utility(self):
        return np.array(self.utility).sum()

    def get_log(self):
        return self.log_bids, self.log_slots

## Plot function

In [ ]:
def plot_cumulative_utility(log_utility, bidders, name_file):
    possible_linestyle = [ "dotted", "dashed", "solid" ]
    n = 4
    plt.figure(figsize=(10, 6))
    for i in range(len(bidders)):
        plt.plot(log_utility[i], label=bidders[i].get_name(), color=bidders[i].get_color(), linestyle = possible_linestyle[int(i/n)])
    plt.xlabel('Round')
    plt.ylabel('Cumulative Utility')
    plt.title('Cumulative Utility for Each Bidder')
    plt.legend()
    plt.grid(True)
    plt.savefig("./plots/req4/" + str(name_file) + " cumulative_utility.png")
    plt.show()

In [ ]:
def plot_cumulative_regret(cumulative_regret, bidders, name_file):
    possible_linestyle = [ "dotted", "dashed", "solid" ]
    n = 4
    plt.figure(figsize=(10, 6))
    for i in range(len(bidders)):
        plt.plot(cumulative_regret[i, :], label=bidders[i].get_name(), color=bidders[i].get_color(), linestyle = possible_linestyle[int(i/n)])
    plt.xlabel('Round')
    plt.ylabel('Cumulative Regret')
    plt.title('Cumulative Regret')
    plt.legend()
    plt.grid(True)
    plt.savefig("./plots/req4/" + str(name_file) + " cumulative_regret.png")
    plt.show()

In [ ]:
def plot_per_round_utility(bidders, name_file):
    num_bidders = len(bidders)
    num_cols = 4  # Adjust this number based on how many plots per row you want
    num_rows = (num_bidders + num_cols - 1) // num_cols  # Calculate the number of rows needed

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 4))
    axes = axes.flatten()  # Flatten the axes array for easy iteration


    for i in range(num_bidders):
        axes[i].plot(bidders[i].utility, color=bidders[i].get_color())
        axes[i].set_title(bidders[i].get_name())
        axes[i].set_xlabel('Round')
        axes[i].set_ylabel('Per-Round Utility')
        axes[i].grid(True)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.savefig("./plots/req4/" + str(name_file) + " per_round_utility.png")
    plt.show()

In [ ]:
def plot_winning_slots(bidders, lambdas, name_file):
    num_bidders = len(bidders)
    num_cols = 4  # Adjust this number based on how many plots per row you want
    num_rows = (num_bidders + num_cols - 1) // num_cols  # Calculate the number of rows needed

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 4))
    axes = axes.flatten()  # Flatten the axes array for easy iteration

    for i in range(num_bidders):
        _, winning_slots = bidders[i].get_log()
        axes[i].scatter(range(len(winning_slots)),lambdas[winning_slots], alpha =0.3, color=bidders[i].get_color())
        axes[i].set_title(bidders[i].get_name())
        axes[i].set_xlabel('Round')
        axes[i].set_ylabel('Winning Slot')
        axes[i].grid(True)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.savefig("./plots/req4/" + str(name_file) + " winning slot.png")
    plt.show()

In [ ]:
def plot_bidding(log_bids, bidders, name_file):
    possible_linestyle = [ "dotted", "dashed", "solid" ]
    n = 4
    plt.figure(figsize=(10, 6))
    for i in range(len(bidders)):
        plt.plot(log_bids[i], label=bidders[i].get_name(), color=bidders[i].get_color(), alpha = 0.5, linestyle = possible_linestyle[int(i/n)])
    plt.xlabel('Round')
    plt.ylabel('Bid')
    plt.title('Bid for each Bidder')
    plt.legend()
    plt.grid(True)
    plt.savefig("./plots/req4/" + str(name_file) + " bid_for_each_round.png")
    plt.show()

## Solver Class

In [ ]:
class Requirement4:
    def __init__(self, params):
        type_of_bidders = 4
        # Parameters of the problem
        self.n_users = params["n_users"]
        self.lambdas = np.sort(np.array(params["lambdas"]))[::-1]
        self.valuations = params["valuations"]
        self.budget = params["budget"]
        self.bidders_per_type = len(self.valuations)
        self.ad_qualities = np.ones(self.bidders_per_type * type_of_bidders )
        self.name_file = params["name_file"]

    def run(self):

        bids_set =  np.linspace(0,1,101) # np.linspace(0,1,int(self.n_users**(1/3)))
        bidders = []
        bids_log = []
        n_slots = len(self.lambdas)

        for i in range( self.bidders_per_type ):
            bidders.append(UCBBiddingAgentExpert( self.valuations[i], self.budget, self.n_users ))
            bidders.append(UCBBiddingAgentExpertUpdateRho( self.valuations[i], self.budget, self.n_users ))
            bidders.append(MultiplicativePacingBiddingAgent(self.valuations[i], self.budget, self.n_users))
            bidders.append(FFMultiplicativePacingBiddingAgent(self.valuations[i], self.budget, self.n_users))

        n_bidders = len(bidders)
        actual_utilities = np.zeros((self.n_users, n_bidders))

        auction = GeneralizedFirstPriceAuction(self.ad_qualities, self.lambdas)


        for u in range(self.n_users):
            bids = []
            for bidder in bidders:
                bids.append(bidder.bid())

            bids_log.append(bids)
            winners, payment_per_click = auction.round(bids)
            payment = payment_per_click*self.lambdas

            for i,bidder in enumerate(bidders):
                rank = find_index(winners, i)
                c_t = 0
                if rank != -1:
                    c_t = payment[rank]
                m_t = np.delete(bids, i)
                m_t = np.sort(m_t)[::-1]
                m_t = m_t[:n_slots]

                bidder.update(self.lambdas,rank,c_t,m_t)

        print("Finished bidding rounds")

        # Instantiate the clairvoyants
        bids_log = np.array(bids_log)
        bids_log.reshape(self.n_users,n_bidders)

        clvoy_per_round_utility = []

        for i, b in enumerate(bidders):
            m_t = np.delete(bids_log, i, axis=1)
            m_t = np.sort(m_t, axis=1)[:,::-1]
            m_t = m_t[:,:n_slots]

            available_bids = np.linspace(0,b.valuation,101)
            win_prob =  np.array([np.sum(bd > m_t, axis=0)/self.n_users for bd in available_bids])
            diff_prob = win_prob[:,1:]-win_prob[:,:-1]
            win_prob = np.append(win_prob[:,:1], diff_prob, axis=1)
            avg_lambdas = np.dot(win_prob,self.lambdas)

            f = (b.valuation-available_bids)*avg_lambdas
            ct = available_bids*avg_lambdas
            rho = self.budget/self.n_users

            gamma, per_round_utility = solve_linear_program(f, ct, rho)
            clvoy_per_round_utility.append(per_round_utility)

        print("\nFinished computing clairvoyant")

        print("\nAgents' utility")
        for i, bidder in enumerate(bidders):
            print(i, bidder.get_utility(), clvoy_per_round_utility[i]*self.n_users)

        log_utility = []
        for b in bidders:
            log_utility.append(b.utility)
        log_utility = np.array(log_utility)
        log_cum_utility = log_utility.cumsum(axis=1)

        clvoy_utility = np.ones(( n_bidders ,self.n_users)) * np.reshape(clvoy_per_round_utility,(n_bidders,1))
        clvoy_cum_utility = clvoy_utility.cumsum(axis=1)

        # Compute cumulative regret for each bidder
        per_round_regret = clvoy_utility - log_utility
        cumulative_regret = np.cumsum(per_round_regret, axis=1)

        #np.savetxt("utility.csv", log_utility)
        #np.savetxt("clvoy.csv", clvoy_utility)
        #np.savetxt("regret.csv", per_round_regret)
        #np.savetxt("cum_regret.csv", cumulative_regret)

        print("\nCumulative Regret:")
        for i in range(len(bidders)):
            print(f"Bidder {i}: {cumulative_regret[i, -1]}")

        print(bids_log.shape)

        # Plot
        plot_cumulative_utility(log_cum_utility, bidders, self.name_file)
        plot_cumulative_regret(cumulative_regret, bidders, self.name_file)
        plot_per_round_utility(bidders, self.name_file)
        plot_winning_slots(bidders, np.append(self.lambdas,0), self.name_file)
        plot_bidding(np.transpose(bids_log), bidders, self.name_file)

## Solving for various Setups 

### Simple case

In [ ]:
n_users = 1000
lambdas = [1] # 3 slots
valuations = [1]
rho = 0.4
budget = n_users * rho

problem_params = {"n_users" : n_users, "lambdas" : lambdas,
                 "budget" : budget, "valuations" : valuations,
                 "name_file" : "simple_case"}

random.seed(42)

req = Requirement4(problem_params)
req.run()

### Standard case

In [ ]:
n_users = 1000
lambdas = [0.8, 0.9, 1] # 3 slots
valuations = [0.8, 0.85, 0.9]
rho = 0.25
budget = n_users * rho

problem_params = {"n_users" : n_users, "lambdas" : lambdas,
                 "budget" : budget, "valuations" : valuations,
                 "name_file" : "standard_case"}

random.seed(42)

req = Requirement4(problem_params)
req.run()

### Low budget - diffrent valuations

In [ ]:
n_users = 1000
lambdas = [0.8, 0.9, 1] # 3 slots
valuations = [0.3, 0.6, 0.9]
rho = 0.1
budget = n_users * rho

problem_params = {"n_users" : n_users, "lambdas" : lambdas,
                 "budget" : budget, "valuations" : valuations,
                 "name_file" : "different_valuations"}

random.seed(42)

req = Requirement4(problem_params)
req.run()

### High budget

In [ ]:
n_users = 1000
lambdas = [0.8, 0.9, 1] # 3 slots
valuations = [0.8, 0.85, 0.9]
rho = 0.5
budget = n_users * rho

problem_params = {"n_users" : n_users, "lambdas" : lambdas,
                 "budget" : budget, "valuations" : valuations,
                 "name_file" : "high_budget"}

random.seed(137)

req = Requirement4(problem_params)
req.run()

### Very high budget

In [ ]:
n_users = 1000
lambdas = [0.8, 0.9, 1] # 3 slots
valuations = [0.8, 0.85, 0.9]
rho = 0.75
budget = n_users * rho

problem_params = {"n_users" : n_users, "lambdas" : lambdas,
                 "budget" : budget, "valuations" : valuations,
                 "name_file" : "very_high_budget"}

random.seed(137)

req = Requirement4(problem_params)
req.run()

### Many Slots

In [ ]:
n_users = 1000
lambdas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
valuations = [0.8, 0.85, 0.9]
rho = 0.25
budget = n_users * rho

problem_params = {"n_users" : n_users, "lambdas" : lambdas,
                 "budget" : budget, "valuations" : valuations,
                 "name_file" : "many_slots"}

random.seed(313)

req = Requirement4(problem_params)
req.run()

### Single Slot

In [ ]:
n_users = 1000
lambdas = [1]
valuations = [0.8, 0.85, 0.9]
rho = 0.25
budget = n_users * rho

problem_params = {"n_users" : n_users, "lambdas" : lambdas,
                 "budget" : budget, "valuations" : valuations,
                 "name_file" : "single_slots"}

random.seed(69)

req = Requirement4(problem_params)
req.run()